In [6]:
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
stock = fetch_openml(name='analcatdata_dmft')
X, y = stock.data, stock.target
X.shape, y.shape
X[0], y[0]

(array([6., 3., 1., 0.]), 'Health_education')

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_validate
import numpy as np 

for i in range(1,10):
    # KNN
    KNNModel = KNeighborsRegressor(i)
    knn_scores = cross_validate(KNNModel, X, y,scoring=make_scorer(mean_squared_error, greater_is_better=False, squared=False))
    mean = np.mean(knn_scores['test_score'])

    # Logistic Regression
    # regressionModel = LogisticRegression()
    # regressionModel.fit(X, y)
    

    print (mean)

TypeError: unsupported operand type(s) for /: 'str' and 'int'